In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [3]:
dataset=pd.get_dummies(dataset,drop_first=True)
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,True
1,15810944,35,20000,0,True
2,15668575,26,43000,0,False
3,15603246,27,57000,0,False
4,15804002,19,76000,0,True
...,...,...,...,...,...
395,15691863,46,41000,1,False
396,15706071,51,23000,1,True
397,15654296,50,20000,1,False
398,15755018,36,33000,0,True


In [4]:
dataset=dataset.drop("User ID",axis=1)

In [5]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [6]:
dataset.columns

Index(['Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [7]:
indep=dataset[['Age', 'EstimatedSalary','Gender_Male']]
dep=dataset[['Purchased']]

In [8]:
indep.shape

(400, 3)

In [9]:
dep.shape

(400, 1)

In [10]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(indep,dep,test_size=0.30,random_state=0)

In [11]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [12]:
from sklearn.linear_model import LogisticRegression

In [13]:
from sklearn.model_selection import GridSearchCV
paran_grid={'solver':['newton-cg','lbfgs','liblinear','saga'],
            'penalty':['l2']}
grid=GridSearchCV(LogisticRegression(),paran_grid,refit=True, verbose=3,n_jobs=-1,scoring='f1_weighted')
grid.fit(X_train,Y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'penalty': ['l2'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'saga']},
             scoring='f1_weighted', verbose=3)

In [14]:
# print best patameter after tuning
# print (grid.best_params)
re=grid.cv_results_
print(re)
grid_prediction=grid.predict(X_test)


{'mean_fit_time': array([0.01733537, 0.01891232, 0.01035485, 0.00781035]), 'std_fit_time': array([0.00857913, 0.01315376, 0.00337138, 0.00281621]), 'mean_score_time': array([0.01364393, 0.01580014, 0.01821527, 0.01451745]), 'std_score_time': array([0.00295117, 0.00433125, 0.00400907, 0.00178177]), 'param_penalty': masked_array(data=['l2', 'l2', 'l2', 'l2'],
             mask=[False, False, False, False],
       fill_value='?',
            dtype=object), 'param_solver': masked_array(data=['newton-cg', 'lbfgs', 'liblinear', 'saga'],
             mask=[False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'penalty': 'l2', 'solver': 'newton-cg'}, {'penalty': 'l2', 'solver': 'lbfgs'}, {'penalty': 'l2', 'solver': 'liblinear'}, {'penalty': 'l2', 'solver': 'saga'}], 'split0_test_score': array([0.83598456, 0.83598456, 0.83598456, 0.83598456]), 'split1_test_score': array([0.80239851, 0.80239851, 0.80239851, 0.80239851]), 'split2_test_score': array([0.6445993,

In [15]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(Y_test,grid_prediction)

In [16]:
from sklearn.metrics import classification_report
clf_report=classification_report(Y_test,grid_prediction)

In [17]:
from sklearn.metrics import f1_score
f1_macro=f1_score(Y_test,grid_prediction,average="weighted")
print("The f1_macro value for best parameter{}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter{'penalty': 'l2', 'solver': 'newton-cg'}: 0.8906190214115365


In [18]:
print(cm)

[[74  5]
 [ 8 33]]


In [19]:
print(clf_report)

              precision    recall  f1-score   support

           0       0.90      0.94      0.92        79
           1       0.87      0.80      0.84        41

    accuracy                           0.89       120
   macro avg       0.89      0.87      0.88       120
weighted avg       0.89      0.89      0.89       120



In [23]:
from sklearn.metrics import roc_auc_score
roc_auc_score(Y_test,grid.predict_proba(X_test)[:,1])

0.9481321395492436

In [25]:
table=pd.DataFrame.from_dict(re)

In [26]:
table


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_penalty,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.017335,0.008579,0.013644,0.002951,l2,newton-cg,"{'penalty': 'l2', 'solver': 'newton-cg'}",0.835985,0.802399,0.644599,0.927778,0.890114,0.820175,0.097839,1
1,0.018912,0.013154,0.015800,0.004331,l2,lbfgs,"{'penalty': 'l2', 'solver': 'lbfgs'}",0.835985,0.802399,0.644599,0.927778,0.890114,0.820175,0.097839,1
2,0.010355,0.003371,0.018215,0.004009,l2,liblinear,"{'penalty': 'l2', 'solver': 'liblinear'}",0.835985,0.802399,0.644599,0.927778,0.890114,0.820175,0.097839,1
3,0.007810,0.002816,0.014517,0.001782,l2,saga,"{'penalty': 'l2', 'solver': 'saga'}",0.835985,0.802399,0.644599,0.927778,0.890114,0.820175,0.097839,1
